## Data Science Immesive Final Project (General Assembly ATX Spring 2017)
The data for my final project was obtained from the Appalachian Trail Conservancy (Harpers Ferry,WV).

### Final Project Question: Can you predict how long it will take a thru hike to complete the Appalachian Trail based on the features of this dataset?



Ideas to concatenate this dataset with weather

# Data Cleaning Table of Content: 
I explore the data column by column to understand what sort of missing or inaccurate values are contained.

def (clean_1): Columns that can/will be dropped
- Rename columns to be more accurate.
- Drop 'Days Elapsed' column as it does not accuratly calculate the number of days on trail. (I replace this column in clean_2 with 'Total_Days')
- Drop 'Hiking_Disability' Column as it was not well reported
- Drop Dog Weight Column (16238 of possible 17835 values were 0)
- Drop columns with too many null values (columns that exceed 85 percent null values)
- drop null values out of columns
- **** come back to this Drop Under2kAmount column since (16125 out of 17835) values were 0 essentially the same as null values 
- Dropped 'Hiking 


def (clean_2/clean_3): Clean up the Age column 
- Fill null values in 'Age' column with median age.
- All values below 4(There were some negative values) 
- The youngest person to thru-hike the AT is 5 so I assume all values above this are 

 
    
def (clean_4): Datetime objects, 
-  Convert 'Start' and 'Finish' columns to Datetime objects
-  Create a new column 'Total Days' that calculates total days on trail (at['Finish] - at['Start'])) 
-  There was as single null value in completion year, however, the finish date showed the hiker completed in 2016 so I changed the value to 2016 
- I conveted the completion yr column from floats to ints 
- converted completion yr to a datetime object



def (clean_5/clean_6): Clean 'Sex' column
- In (clean_1) I dropped all null values in this column 
- Drop 'oher' and 'othe' out of the sex column
- Convert the 'Sex' column to numerical format with the map function
- Drop null values in the Sex column. There were only 2. 
- Converted M/F from float to int

def (clean_7/clean_8): Clean 'Hike_Type' column (Drop section, alternative hikes, Yo Yo)
- Drop values in the Hike column that are labeled alternative as I am unsure what these are and how to define them.
- Since I am only interested in Thru Hikers and trying to predict time on trail based on people trying to hike in 6 months, and since all thru section hikers take varying amounts of time. 
- Difficult to estimate value for a Yo Yo so I dropped it, it was only a single column

def (clean_9): Categorize the race column:
- 'Prefer not to say' = 0
- []
    


def (clean_10): Reset Index 
- The index needed to be reset since I had dropped many rows containing null values 



# To do 

3.) Make Boolean columns into dummy variables 

5.) Yo Yo split that into 1 Northbound and 1 southbound. 

6.) Race 

7.) State 

8.) There are negative total days (str.replace values)




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
at = pd.read_excel('BenHikersInfo.xlsx')

## Cleaning Dataset

In [3]:
def clean_1(df):
    df.drop(labels = df[['Disease Other','School','Hiking Disability','DogWeightLbs','Days Elapsed','MinorityComments','School',
                         'DogBreed','Disability','Under2KAmount']], axis = 1, inplace = True)
    df.rename(columns = {'AgeYrFin': 'Age','Race or Heritage': 'Race',
                        'StAbbrev':'State', 'Hiking Disability ': 'Hiking_Disability', 
                         'Hike Type': 'Hike_Type', 'Yr Comp':'completion_yr',
                         },inplace = True)
    df.dropna(subset= ['Start','Sex','Race','Country','State','Hike_Type'],inplace = True)

clean_1(at)  


def clean_2(df):
    df['Age'] = df['Age'].fillna(df['Age'].median())
clean_2(at)


def clean_3(x,median_age): 
    if x < 4:
        return median_age
    else:
        return x 
at['Age'] = at['Age'].apply(lambda x: clean_3(x, at['Age'].median()))

In [4]:
def clean_4(df):
    df['Finish'] = pd.to_datetime(df['Finish'])
    df['Start'] = pd.to_datetime(df['Start'])
    df['Total_Days'] = df['Finish'] - df['Start']
    df['completion_yr'].fillna(2016,inplace = True)
    df['completion_yr'] = df['completion_yr'].astype(int)
clean_4(at)

In [5]:
def clean_5(df):
    df['Sex'] = df['Sex'].map({'M':0,'F':1})
    to_drop = ['othe','Oher']
    x = df[df['Sex'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_5(at)

def clean_6(df):
    df.dropna(subset = ['Sex'], inplace = True)
    df['Sex'] = df['Sex'].astype(int)
clean_6(at)

In [6]:
def clean_7(df):
    to_drop = ['Alternative','alternative','Alterntive', 'Yo Yo']
    x = df[df['Hike_Type'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_7(at)

def clean_8(df):
    to_drop = ['Section']
    x = df[df['Hike_Type'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_8(at)

In [35]:
def clean_9(df):
    # Prefer not to say: 0
    df['Race'] = df['Race'].replace(['Prefer not to say'], 0)
    
    # Caucasion Variation: 1
    df['Race'] = df['Race'].replace(['White','WASP','Irish','Hewbrew/White','English','Caucasian / Viking',
                                    'Caucasian or White','Caucasian or White/Irish','Caucasian / Polish / Irish',
                                    'Caucasian / South Africaan','Caucasian or White/Amish','Caucasian / Irish'
                                     'Caucasian or White/Jewish','Caucasian / Irish',
                                     'Anglo Saxon / Scottish / American','Caucasian or White (Jewish)',
                                    ], 1)
    # African American Variation: 2
    df['Race'] = df['Race'].replace(['African American','African American/Biracial',], 2)
    # Asian Variation: 3
    df['Race'] = df['Race'].replace(['Asian','Chinese','Asian / Chinese','Vietnamese','Thai',
                                    'Taiwanese / American'], 3)
    # Latino Variation: 4
    df['Race'] = df['Race'].replace(['Hispanic','Latino / Venzuelan',], 4)
    # Native American Variation: 5
    
    # Mixed Race: 6
    df['Race'] = df['Race'].replace(['Mixed','mixed','Caucasian / Black / West Indian','Hispanic/Black',
                                    ], 6)
    
    # Race Not on Application: 7 
    df['Race'] = df['Race'].replace(['No Application - Letter Only','Not on Application' ], 7)
    
    # Other: 8 
    df['Race'] = df['Race'].replace(['Viking','Other'], 8)
    
     

clean_9(at)

In [44]:
from collections import Counter
Counter(at['Race'])

Counter({0: 6860,
         1: 5561,
         2: 18,
         3: 40,
         4: 37,
         6: 5,
         7: 2,
         'European/Native American': 1,
         'Cherokee & English': 1,
         'Caucasian or White/Pacific Islander': 1,
         'European/American': 1,
         'American Indian / Caucasian': 7,
         'Romanian': 1,
         'Irish/American': 1,
         'Hispanic/Black': 1,
         'British / American': 1,
         'Swiss': 1,
         'German/Irish American': 1,
         'Swiss/American': 1,
         'Jewish/White': 1,
         'Western European / American': 1,
         'African / European / Carribean': 1,
         'European / American': 1,
         'Asian / American': 3,
         'Caucasian / Italian / Eastern European': 1,
         'Mexican / American': 1,
         'Japanese-American': 1,
         'German': 3,
         'Hispanic/German': 1,
         'Caucasian/Hispanic': 2,
         'Canadian': 1,
         'Western European': 1,
         'Irish/Italian': 1,
  

In [37]:
def clean_10(df):
    df.reset_index(inplace = True, drop = True)
clean_10(at)

In [42]:
at.head(50)


,Finish,Age,Race,State,Sex,Country,Hike_Type,Start,completion_yr,HikedWDog,WinterHike,Under2KSpent,SchoolCredit,Minority,Vegetarian,DehydrateOwnFood,Giardia,Lyme,Norovirus,Total_Days
0,1948-12-31,29.0,0,PA,0,USA,NOBO,1948-01-01,1948,False,False,False,False,False,False,False,False,False,False,365 days
1,1951-10-10,28.0,0,CT,0,USA,SOBO,1951-05-30,1951,False,False,False,False,False,False,False,False,False,False,133 days
2,1951-09-30,29.0,0,GA,0,USA,NOBO,1951-05-31,1951,False,False,False,False,False,False,False,False,False,False,122 days
3,1959-09-17,20.0,1,GA,0,USA,NOBO,1959-05-23,1959,False,False,False,False,False,False,False,False,False,False,117 days
4,1965-09-23,29.0,0,PA,0,USA,SOBO,1965-04-16,1965,False,False,False,False,False,False,False,False,False,False,160 days
5,1965-10-25,46.0,0,PA,0,USA,SOBO,1965-07-19,1965,False,False,False,False,False,False,False,False,False,False,98 days
6,1967-09-30,29.0,0,PA,0,USA,NOBO,1967-04-05,1967,False,False,False,False,False,False,False,False,False,False,178 days
7,1968-12-31,29.0,0,CT,0,USA,NOBO,1968-01-01,1968,False,False,False,False,False,False,False,False,False,False,365 days
8,1969-08-05,38.0,0,AK,0,USA,NOBO,1969-02-28,1969,False,False,False,False,False,False,False,False,False,False,158 days
9,1969-08-05,29.0,0,AK,1,USA,NOBO,1969-02-28,1969,False,False,False,False,False,False,False,False,False,False,158 days


In [11]:
at['Race'].str.replace()

TypeError: replace() missing 2 required positional arguments: 'pat' and 'repl'

def (clean_9): Categorize the race column:
- 'Prefer not to say' = 0
- Caucasion variation = 1
- African American Variation = 2
- Asian Variation = 3
- Hispanic Variation = 4
- 
    

In [ ]:
# Drop anything that is not a clearly identifiable race (i.e American is not a race)
def clean_9(df):
    df['Race'] = df['Race'].str.replace('Prefer not to say', '0')

clean_9(at)
        

In [ ]:
at['Race'].value_counts()

In [ ]:
at.head()

In [ ]:
#plt.figure(figsize = (10,10))
#null.plot(kind = 'bar')

In [ ]:
# Exploratory analysis

In [ ]:
# Null values by column 
#plt.figure(figsize = (10,10))
#z = at.isnull().sum()
#z.plot(kind = 'bar')